In [1]:
from autogen_agentchat.agents import CodingAssistantAgent, AssistantAgent
from autogen_agentchat.task import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import AzureOpenAIChatCompletionClient

import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')

In [3]:
client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True,
    }
)

In [4]:
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are a helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are a helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are a helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed tfinal travel plan. You must ensure th b at the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)

In [5]:
termination = TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent],
    termination_condition=termination
)

In [6]:
result = await group_chat.run(task="Plan a 3 day trip to Paris.")

In [7]:
from IPython.display import Markdown, display

total_prompt_tokens = 0
total_completion_tokens = 0

for index, message in enumerate(result.messages, start=1):
    display(Markdown('<hr>'))
    text = f"## Step: {index} (output from {message.source})"
    display(Markdown(text))

    if message.models_usage:
        print(f'Prompt Tokens: {message.models_usage.prompt_tokens}')
        print(f'Completion Tokens: {message.models_usage.completion_tokens}')
        total_prompt_tokens += message.models_usage.prompt_tokens
        total_completion_tokens += message.models_usage.completion_tokens
    display(Markdown('<hr>'))

    display(Markdown(message.content))

display(Markdown('<hr>'))
print(f'Total Prompt Tokens: {total_prompt_tokens}')
print(f'Total Completion Tokens: {total_completion_tokens}')
print(f'Total Tokens: {total_prompt_tokens + total_completion_tokens}')

<hr>

## Step: 1 (output from user)

<hr>

Plan a 3 day trip to Paris.

<hr>

...

Prompt Tokens: 1948
Completion Tokens: 844


<hr>

Here's a comprehensive and integrated travel plan for your three-day trip to Paris, considering historical landmarks, cultural experiences, local insights, and language tips to enhance your journey:

---

### Day 1: Historical and Cultural Insights

**Morning:**

- **Notre-Dame Cathedral and Île de la Cité:** Begin your trip at the heart of Paris with a visit to the iconic Notre-Dame Cathedral. Though it's closed for restoration, admire its Gothic architecture from the outside, and wander the charming Île de la Cité.

- **Sainte-Chapelle:** A short walk away, discover the remarkable stained-glass windows of this Gothic gem. Arrive early to avoid long queues.

**Afternoon:**

- **Lunch in Le Marais:** Explore the historic district of Le Marais. Enjoy lunch at a cozy café while savoring traditional French cuisine. Afterward, stroll through its narrow streets dotted with boutiques and art galleries.

- **Musée Picasso or Musée Carnavalet:** Depending on your interest, delve into Picasso’s artworks or experience Parisian history.

**Evening:**

- **Dinner near the Louvre:** Choose a restaurant in the Louvre district for dinner, enjoying a variety of culinary options.

- **Evening at the Louvre:** If it’s a Wednesday or Friday, the Louvre stays open late. Use this opportunity to explore masterpieces like the Mona Lisa and Venus de Milo without the daytime crowd.

### Day 2: Iconic Sights and Scenic Views

**Morning:**

- **Eiffel Tower:** Start your day with Paris's iconic landmark. Pre-book your tickets to skip lines and enjoy panoramic views from the summit or second floor.

**Afternoon:**

- **Champs-Élysées and Arc de Triomphe:** Walk the famous avenue and visit the Arc de Triomphe. Climb to its top for a different city perspective. Lunch at a café on Champs-Élysées, tasting classic Parisian fare.

- **Les Invalides:** Explore this monument complex, including museums and Napoleon’s Tomb.

**Evening:**

- **Seine River Cruise:** Experience Paris at sunset with a scenic Seine River cruise. The illuminated city offers a captivating view.

- **Montmartre:** Head to Montmartre for dinner. Dive into its bohemian charm, visit the Sacré-Cœur Basilica for a breathtaking view, and dine in a local bistro.

### Day 3: Artistic and Local Flavors

**Morning:**

- **Musée d'Orsay:** Visit the former railway station to appreciate its vast collection of Impressionist and Post-Impressionist art. Marvel at works by Monet, Van Gogh, and Degas.

**Afternoon:**

- **Lunch in Saint-Germain-des-Prés:** Enjoy a relaxing lunch in this historic district, known for its literary cafés and lively ambiance.

- **Luxembourg Gardens:** Stroll or unwind in the enchanting gardens, a serene escape from the urban bustle.

**Evening:**
  
- **Le Marais for Dinner:** Spend your last evening dining in Le Marais. Enjoy classic French dishes at one of its many charming restaurants.

- **Optional Cabaret Show:** Consider ending your Parisian adventure with a cabaret show at the Moulin Rouge or Le Lido for an unforgettable night. Book in advance to secure seats.

---

### Additional Tips and Language Insights:

- **Accommodation:** Stay in central areas like Le Marais, Latin Quarter, or Saint-Germain-des-Prés for convenient access to attractions.

- **Transportation:** Utilize the Paris Métro with a Paris Visite or Navigo Easy pass for hassle-free travel.

- **Basic French Phrases:** Use simple phrases like "Bonjour" (hello), "Merci" (thank you), and "Parlez-vous anglais?" (Do you speak English?) to enhance interactions.

- **Translation Tools:** Carry a translation app to help with language gaps during your trip.

- **Dining Etiquette:** Reserve tables ahead, especially for dinners at popular restaurants.

This plan balances must-see sights with cultural experiences and local flavors, ensuring you have an enriching and memorable visit to Paris. Enjoy your trip to the City of Light! 

TERMINATE

<hr>

Total Prompt Tokens: 4406
Total Completion Tokens: 2681
Total Tokens: 7087
